# 04a - Binary Class Federated Learning with Edge IIoT Dataset using Flower and TensorFlow/Keras

In this notebook we use the Flower Federated Learning library (flower.dev) with TensorFlow/Keras to distribute the Edge-IIoT data across multiple clients in various different ways.

In [12]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['INDIVIDUAL_ATTACK', 'ATTACK_GROUP', 'STRATIFIED']
METHOD = 'STRATIFIED'
NUM_OF_STRATIFIED_CLIENTS = 10 # only applies to stratified method
print (METHOD)

STRATIFIED


In [13]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [14]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10


In [15]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.3.0
numpy 1.24.2
torch 1.13.1
torchvision 0.14.1
Training on cuda:0


In [16]:
dataset_path = "../datasets/Edge-IIoT/"

df = pd.read_pickle(dataset_path + "Edge-IIoTset dataset/Selected dataset for ML and DL/ML-EdgeIIoT-dataset.pkl")   

## Make dataframes for Binary classification 

In [17]:
# Binary attack dataframe
binary_df=df[['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4',
       'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum',
       'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused',
       'http.file_data', 'http.content_length', 'http.request.uri.query',
       'http.request.method', 'http.referer', 'http.request.full_uri',
       'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack',
       'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin',
       'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack',
       'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options',
       'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream',
       'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu',
       'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request',
       'dns.retransmit_request_in', 'mqtt.conack.flags',
       'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len',
       'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.msgtype', 'mqtt.proto_len',
       'mqtt.protoname', 'mqtt.topic', 'mqtt.topic_len', 'mqtt.ver',
       'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id', 'Attack_label']]

In [19]:
known_sensor_ip_addresses = [ '192.168.0.101', '192.168.2.194', '192.168.3.18', '192.168.4.73', '192.168.5.47', '192.168.6.56', '192.768.7.62', '192.168.8.163']
print ("known_sensor_ip_addresses:", known_sensor_ip_addresses)

tcp_dos_attack_ip_addresses = [ '207.192.25.133', '94.196.109.185', '133.149.252.77', '220.146.94.148' ]
tdp_dos_atack_ip_addresses = [ '190.123.219.128', '16.226.184.201', '153.125.214.15', '91.184.12.91' ]
http_attack_ip_addresses = [ '192.168.0.170', '216.58.198.74' ]
icmp_flood_attack_ip_addresses = [ '213.117.18.213', '183.223.100.122', '166.153.227.121', '49.81.59.152', '227.117.33.125' ]
port_scan_attack_ip_addresses = [ '192.168.0.170' ]
os_fingerprinting_attack_ip_addresses = [ '192.168.0.170' ]
vuln_scan_attack_ip_addresses = [ '192.168.0.170', '142.250.200.205', '172.217.19.35', '142.250.201.10' ]
dns_spoof_attack_ip_addresses = [ '192.168.0.101', '192.168.0.152', '172.217.19.35', '192.168.0.170' ]
arp_spoof_attack_ip_addresses = [ '192.168.0.101', '192.168.0.152', '172.217.19.35', '192.168.0.170' ]
xss_attack_ip_addresses = [ '192.168.0.170', '172.217.19.42', '104.16.87.20' ]
sql_injection_attack_ip_addresses = [ '192.168.0.170' ]
upload_attack_ip_addresses = [ '192.168.0.170' ]
backdoor_attack_ip_addresses = [ '192.168.0.170' ]
password_attack_ip_addresses = [ '192.168.0.170' ]
ransomware_attack_ip_addresses = [ '192.168.0.170' ] 

# Combine all attack IP addresses into one list, ensuring no duplicates
known_attacker_ip_addresses = list(set(tcp_dos_attack_ip_addresses + tdp_dos_atack_ip_addresses + http_attack_ip_addresses + icmp_flood_attack_ip_addresses + port_scan_attack_ip_addresses + os_fingerprinting_attack_ip_addresses + vuln_scan_attack_ip_addresses + dns_spoof_attack_ip_addresses + arp_spoof_attack_ip_addresses + xss_attack_ip_addresses + sql_injection_attack_ip_addresses + upload_attack_ip_addresses + backdoor_attack_ip_addresses + password_attack_ip_addresses + ransomware_attack_ip_addresses))
print (f"known_attacker_ip_addresses: \nNumber of IPs {len(known_attacker_ip_addresses)}\n{known_attacker_ip_addresses}")


known_sensor_ip_addresses: ['192.168.0.101', '192.168.2.194', '192.168.3.18', '192.168.4.73', '192.168.5.47', '192.168.6.56', '192.768.7.62', '192.168.8.163']
known_attacker_ip_addresses: 
Number of IPs 22
['172.217.19.35', '133.149.252.77', '192.168.0.101', '192.168.0.170', '172.217.19.42', '142.250.200.205', '16.226.184.201', '166.153.227.121', '213.117.18.213', '91.184.12.91', '220.146.94.148', '142.250.201.10', '104.16.87.20', '153.125.214.15', '190.123.219.128', '183.223.100.122', '227.117.33.125', '207.192.25.133', '216.58.198.74', '192.168.0.152', '94.196.109.185', '49.81.59.152']


With the exception of DDOS attacks, the attacks mainly come from a small subset of IP addresses. `192.168.0.170` being responsible for a lot of the attacks. From the data exploration workbook `02b-ML-Data-Exploration.ipynb` we can also see that the attacked IP is always the `192.168.0.128` edge server. This means it is not feasible the divide the traffic either by attacker or attacked IP address.

## Binary Classification

Categorical data encoding (Dummy Encoding):

EG. Takes a product category and converts it to a binary vector

In [20]:
def encode_text_dummy(df, name):

    dummies = pd.get_dummies(df[name])

    for x in dummies.columns:

        dummy_name = f"{name}-{x}"

        df[dummy_name] = dummies[x]

    df.drop(name, axis=1, inplace=True)

encode_text_dummy(binary_df,'http.request.method')

encode_text_dummy(binary_df,'http.referer')

encode_text_dummy(binary_df,"http.request.version")

encode_text_dummy(binary_df,"dns.qry.name.len")

encode_text_dummy(binary_df,"mqtt.conack.flags")

encode_text_dummy(binary_df,"mqtt.protoname")

encode_text_dummy(binary_df,"mqtt.topic")

In [21]:
# print max index value of binary_df
print("max index value of binary_df:", max(binary_df.index))
print(binary_df.shape)

max index value of binary_df: 157799
(157800, 90)


We need to drop some unrequired columns from the DF, but we need to keep the original around as we may need to split the data differently for different models based on things like the IP address

In [22]:
#drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
drop_columns = ["frame.time", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 

         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",

         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",

         "tcp.dstport", "udp.port", "mqtt.msg"]

binary_df = binary_df.drop(drop_columns, axis=1)

binary_df = binary_df.dropna(axis=0, how='any')

binary_df = binary_df.drop_duplicates(subset=None, keep="first")

# We cant shuffle at this point as we need to keep the order so we can split the dataset later based on things like IP address
#binary_df_copy = shuffle(binary_df_copy)

# Compute the number of missing values (NaN or null) in each column of a pandas DataFrame object named df.
binary_df.isna().sum()

ip.src_host                            0
ip.dst_host                            0
arp.opcode                             0
arp.hw.size                            0
icmp.checksum                          0
                                      ..
mqtt.protoname-0.0                     0
mqtt.protoname-MQTT                    0
mqtt.topic-0                           0
mqtt.topic-0.0                         0
mqtt.topic-Temperature_and_Humidity    0
Length: 77, dtype: int64

In [23]:
# print max index value of binary_df
print("max index value of binary_df:", max(binary_df.index))
print(binary_df.shape)

binary_df.reset_index(drop=True, inplace=True)

# print max index value of binary_df
print("max index value of binary_df:", max(binary_df.index))
print(binary_df.shape)


max index value of binary_df: 157799
(153131, 77)
max index value of binary_df: 153130
(153131, 77)


In [24]:

label = binary_df['Attack_label']
le = preprocessing.LabelEncoder()
label_n = le.fit_transform(label.values)

# Stratify based on the attack label to balance the dataset - This is our original copy of the data include IP addresses
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(binary_df, label_n, stratify=label_n, test_size=0.2, random_state=42)

print(X_train_df.shape)

X_train_df.reset_index(drop=True, inplace=True)

#print the max index of X_train_df
print(X_train_df.index.max())

#  print the max index of binary_df
print(binary_df.index.max())

(122504, 77)
122503
153130


In [25]:
label = binary_df['Attack_label']
le = preprocessing.LabelEncoder()
label_n = le.fit_transform(label.values)

# Stratify based on the attack label to balance the dataset - This is our original copy of the data include IP addresses
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(binary_df, label_n, stratify=label_n, test_size=0.2, random_state=42)

print(X_train_df.shape)

#print the max index of X_train_df
print(X_train_df.index.max())

#  print the max index of binary_df
print(binary_df.index.max())

(122504, 77)
153130
153130


In [27]:
binary_df_copy = binary_df.copy()

binary_df_copy = binary_df_copy.drop(["ip.src_host", "ip.dst_host", "Attack_label"], axis=1)

# This is our copy of the data without IP addresses
scaled_features = StandardScaler().fit_transform(binary_df_copy.values)
X_train, X_test, y_train, y_test = train_test_split(scaled_features, label_n, stratify=label_n, test_size=0.2, random_state=42)

print ("Train:", X_train.shape, y_train.shape)
print ("Test:", X_test.shape, y_test.shape)


Train: (122504, 74) (122504,)
Test: (30627, 74) (30627,)


In [28]:
# Binary Attack classification
N_CLASSES = 2
# Number of features = size of binary_df - 1 (Attack_label)
N_FEATURES = binary_df.shape[1] - 1

In [48]:
fl_X_train = []
fl_y_train = []

if METHOD == 'STRATIFIED':
    # Stratfiy the dataset
    from sklearn.model_selection import StratifiedKFold

    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    skf.get_n_splits(X_train, y_train)

    for _, train_index in skf.split(X_train, y_train):
        print("TRAIN:", train_index)
        X_np = X_train[train_index]
        y_np = y_train[train_index]

        fl_X_train.append(X_np)
        fl_y_train.append(y_np)

else: # UNUSED
    # Individual IP address
    for ip in known_sensor_ip_addresses:
        new_ip = [ip]
        print("new_ip:", new_ip)

        X_train_df['ip.src_host']
        
        print("Shape X_Train:", X_train.shape)
        print("Shape y_Train:", y_train.shape)

        # Filter dataframe by IP address
        new_df_src = X_train_df[ X_train_df['ip.src_host'].isin(new_ip) ]
        new_df_dst = X_train_df[ X_train_df['ip.dst_host'].isin(new_ip) ]

        print("Shape new_df_src:", new_df_src.shape)
        print("Shape new_df_dst:", new_df_dst.shape)

        X_np = np.vstack([ X_train[ new_df_src.index, : ], X_train[ new_df_dst.index, :] ])
        y_np = np.hstack([ y_train[ new_df_src.index ], y_train[ new_df_dst.index ] ])

        print ("x_np:", X_np.shape)
        print ("y_np:", y_np.shape)

        fl_X_train.append(X_np)
        fl_y_train.append(y_np)


TRAIN: [     4      5      6 ... 122485 122498 122502]
TRAIN: [     3     13     21 ... 122489 122490 122503]
TRAIN: [     9     26     29 ... 122443 122469 122473]
TRAIN: [    15     23     30 ... 122479 122488 122496]
TRAIN: [    18     19     27 ... 122483 122492 122501]
TRAIN: [     0      8     28 ... 122484 122491 122499]
TRAIN: [    14     17     25 ... 122486 122487 122494]
TRAIN: [    12     24     34 ... 122493 122497 122500]
TRAIN: [     2     10     16 ... 122474 122476 122495]
TRAIN: [     1      7     20 ... 122448 122452 122471]


In [49]:
# Print out the size of the fl_X_train
print ("fl_X_train size:", len(fl_X_train))

# Print out the size of each element in the fl_X_train
for i in range(len(fl_X_train)):
    print ("fl_X_train[", i, "]:", fl_X_train[i].shape)
    print ("fl_y_train[", i, "]:", fl_y_train[i].shape)


fl_X_train size: 10
fl_X_train[ 0 ]: (12251, 74)
fl_y_train[ 0 ]: (12251,)
fl_X_train[ 1 ]: (12251, 74)
fl_y_train[ 1 ]: (12251,)
fl_X_train[ 2 ]: (12251, 74)
fl_y_train[ 2 ]: (12251,)
fl_X_train[ 3 ]: (12251, 74)
fl_y_train[ 3 ]: (12251,)
fl_X_train[ 4 ]: (12250, 74)
fl_y_train[ 4 ]: (12250,)
fl_X_train[ 5 ]: (12250, 74)
fl_y_train[ 5 ]: (12250,)
fl_X_train[ 6 ]: (12250, 74)
fl_y_train[ 6 ]: (12250,)
fl_X_train[ 7 ]: (12250, 74)
fl_y_train[ 7 ]: (12250,)
fl_X_train[ 8 ]: (12250, 74)
fl_y_train[ 8 ]: (12250,)
fl_X_train[ 9 ]: (12250, 74)
fl_y_train[ 9 ]: (12250,)


In [50]:
len(label.unique()) 

2

In [53]:
NUM_OF_CLIENTS = len(fl_X_train)
print("NUM_OF_CLIENTS:", NUM_OF_CLIENTS)    

NUM_OF_ROUNDS = 10

print("Checking data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_Y_train[i].shape)

print("\nDeploy Simulation")


NUM_OF_CLIENTS: 10
Checking data split groups
0 : X Shape (12251, 74) Y Shape (12251,)
1 : X Shape (12251, 74) Y Shape (12251,)
2 : X Shape (12251, 74) Y Shape (12251,)
3 : X Shape (12251, 74) Y Shape (12251,)
4 : X Shape (12250, 74) Y Shape (12250,)
5 : X Shape (12250, 74) Y Shape (12250,)
6 : X Shape (12250, 74) Y Shape (12250,)
7 : X Shape (12250, 74) Y Shape (12250,)
8 : X Shape (12250, 74) Y Shape (12250,)
9 : X Shape (12250, 74) Y Shape (12250,)

Deploy Simulation


# FL Part

In [54]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=1, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=32)
        print ("Client ", self.cid, "Evaluating complete...", accuracy, loss)
        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(256, activation='sigmoid'),
      Dense(128, activation='sigmoid'), 
      #Dense(18, activation='sigmoid'),  
      Dense(len(label.unique()), activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


print ("Deploy simulation...")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print ("Server Evaluating...", eval_count)
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print ("Server Evaluating complete...", accuracy, loss)
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(256, activation='sigmoid'),
      Dense(128, activation='sigmoid'), 
      #Dense(18, activation='sigmoid'),  
      Dense(len(label.unique()), activation='sigmoid')
    ])
server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-07-05 16:48:30,142 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


flwr 1.3.0
numpy 1.24.2
tf 2.11.0
Deploy simulation...


2023-07-05 16:48:37,770	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-07-05 16:48:41,990 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'GPU': 1.0, 'CPU': 24.0, 'memory': 11390255924.0, 'object_store_memory': 5695127961.0}
INFO flwr 2023-07-05 16:48:41,991 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-05 16:48:41,991 | server.py:270 | Requesting initial parameters from one random client
INFO flwr 2023-07-05 16:48:48,512 | server.py:274 | Received initial parameters from one random client
INFO flwr 2023-07-05 16:48:48,513 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=30100) Client ID: 8
Server Evaluating... 0
958/958 [==============================] - 1s 810us/step


INFO flwr 2023-07-05 16:48:50,625 | server.py:91 | initial parameters (loss, other metrics): 1.3268556594848633, {'accuracy': 0.157410129904747}
INFO flwr 2023-07-05 16:48:50,626 | server.py:101 | FL starting
DEBUG flwr 2023-07-05 16:48:50,627 | server.py:215 | fit_round 1: strategy sampled 10 clients (out of 10)


...

Prediction:  [[1.4773601e-05 9.9989861e-01]
 [9.9992716e-01 4.8176833e-05]
 [2.3115503e-05 9.9984580e-01]
 ...
 [2.4804855e-05 9.9983495e-01]
 [1.8158196e-05 9.9987620e-01]
 [1.6918899e-05 9.9988437e-01]] (30627, 2)
Server Evaluating complete... 0.9999673366546631 0.00013965113612357527
(launch_and_evaluate pid=30100) Client ID: 9
(launch_and_evaluate pid=30100) Client  9 Evaluating...
(launch_and_evaluate pid=37908) Client ID: 4
(launch_and_evaluate pid=37908) Client  4 Evaluating...
(launch_and_evaluate pid=24824) Client ID: 5
(launch_and_evaluate pid=24824) Client  5 Evaluating...
(launch_and_evaluate pid=27224) Client ID: 7
(launch_and_evaluate pid=27224) Client  7 Evaluating...
(launch_and_evaluate pid=28356) Client ID: 2
(launch_and_evaluate pid=28356) Client  2 Evaluating...
(launch_and_evaluate pid=30100) 
(launch_and_evaluate pid=30100)   1/383 [..............................] - ETA: 1:00 - loss: 0.0000e+00 - accuracy: 1.0000
(launch_and_evaluate pid=30100)  49/383 [==>.......

DEBUG flwr 2023-07-05 16:49:50,161 | server.py:179 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2023-07-05 16:49:50,161 | server.py:144 | FL finished in 59.5347417999983
INFO flwr 2023-07-05 16:49:50,162 | app.py:202 | app_fit: losses_distributed [(1, 0.0016396539745818324), (2, 0.00010860346247256878), (3, 4.605048730855203e-05), (4, 3.8030443278718894e-05), (5, 1.3393729003230721e-06), (6, 8.208576952433955e-08), (7, 2.5535116396583954e-07), (8, 1.995854907186644e-07), (9, 1.9802135017295806e-07), (10, 2.406117811235709e-07)]
INFO flwr 2023-07-05 16:49:50,163 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-07-05 16:49:50,163 | app.py:204 | app_fit: losses_centralized [(0, 1.3268556594848633), (1, 0.0016874748980626464), (2, 0.0002679765457287431), (3, 0.0001594357454450801), (4, 0.00021414806542452425), (5, 0.00018651386199053377), (6, 0.00019388312648516148), (7, 0.00019091418653260916), (8, 0.00014888093573972583), (9, 0.00016968075942713767), (10, 

(launch_and_evaluate pid=30100) 
(launch_and_evaluate pid=30100) 188/383 [=============>................] - ETA: 0s - loss: 1.9815e-11 - accuracy: 1.0000
(launch_and_evaluate pid=30100) 237/383 [=================>............] - ETA: 0s - loss: 1.5719e-11 - accuracy: 1.0000
(launch_and_evaluate pid=30100) 286/383 [=====================>........] - ETA: 0s - loss: 7.9882e-08 - accuracy: 1.0000
(launch_and_evaluate pid=37908) 
(launch_and_evaluate pid=37908) 195/383 [==============>...............] - ETA: 0s - loss: 3.6298e-10 - accuracy: 1.0000
(launch_and_evaluate pid=37908) 245/383 [==================>...........] - ETA: 0s - loss: 2.8890e-10 - accuracy: 1.0000
(launch_and_evaluate pid=37908) 293/383 [=====================>........] - ETA: 0s - loss: 2.4157e-10 - accuracy: 1.0000
(launch_and_evaluate pid=24824) 
(launch_and_evaluate pid=24824) 192/383 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
(launch_and_evaluate pid=24824) 241/383 [=============

History (loss, distributed):
	round 1: 0.0016396539745818324
	round 2: 0.00010860346247256878
	round 3: 4.605048730855203e-05
	round 4: 3.8030443278718894e-05
	round 5: 1.3393729003230721e-06
	round 6: 8.208576952433955e-08
	round 7: 2.5535116396583954e-07
	round 8: 1.995854907186644e-07
	round 9: 1.9802135017295806e-07
	round 10: 2.406117811235709e-07
History (loss, centralized):
	round 0: 1.3268556594848633
	round 1: 0.0016874748980626464
	round 2: 0.0002679765457287431
	round 3: 0.0001594357454450801
	round 4: 0.00021414806542452425
	round 5: 0.00018651386199053377
	round 6: 0.00019388312648516148
	round 7: 0.00019091418653260916
	round 8: 0.00014888093573972583
	round 9: 0.00016968075942713767
	round 10: 0.00013965113612357527
History (metrics, centralized):
{'accuracy': [(0, 0.157410129904747), (1, 0.9999346733093262), (2, 0.9998694062232971), (3, 0.9999346733093262), (4, 0.999902069568634), (5, 0.9999673366546631), (6, 0.9999673366546631), (7, 0.9999673366546631), (8, 0.999967336

(launch_and_evaluate pid=27224) Client  7 Evaluating complete... 1.0 3.8925480894125286e-11
(launch_and_evaluate pid=28356) Client  2 Evaluating complete... 1.0 2.0694367108831102e-08
